# R-Type Game Generator - AI Creates a Game

This notebook shows how to use aisuite + MCP tools to have an AI generate a complete, playable R-Type game.

**What it does**: The LLM generates an Rtype game in HTML/CSS/JavaScript, saves it to a file using MCP filesystem tools, and we display it right in the notebook.

**Requirements**: `OPENAI_API_KEY` or `ANTHROPIC_API_KEY` in your `.env` file (depending on which model you choose)

## Setup

In [ ]:
import os
import boto3 # Ensure boto3 is installed
#from dotenv import load_dotenv
import aisuite as ai
from aisuite.mcp import MCPClient
from IPython.display import IFrame, display  # For displaying HTML file

#load_dotenv()
# 1. Create your session using the specific profile
session = boto3.Session(profile_name="WMAIDEV", region_name="us-east-1")

# 2. Extract the temporary credentials from that session
creds = session.get_credentials().get_frozen_credentials()

# 3. Inject these into the environment so aisuite's internal boto3 client sees them
os.environ["AWS_ACCESS_KEY_ID"] = creds.access_key
os.environ["AWS_SECRET_ACCESS_KEY"] = creds.secret_key
os.environ["AWS_SESSION_TOKEN"] = creds.token # Essential for SSO/Temporary sessions
os.environ["AWS_DEFAULT_REGION"] = session.region_name

# Initialize filesystem MCP server for file writing
filesystem_mcp = MCPClient(
    command="npx",
    args=["-y", "@modelcontextprotocol/server-filesystem", os.getcwd()]
)

print("✓ Ready with AWS SDK!")



## Craft Instructions

In [ ]:
prompt = """Create a complete, playable R-Type horizontally scrolling shooter video game.

**EXECUTION RULES:**
- Execute ALL tools silently (no intermediate text responses)
- Write the HTML file FIRST, then provide a brief summary

**GAME REQUIREMENTS:**
 Styling:
 - Clean, modern look
   - Centered on page
   - Nice colors (dark background, use similiar colors to the original 1987 game)
   - make the player's shop look like the origninal r-type ship
   - When the score increments by 1000 then power up the players weapon system
   - Randomly include ship shield options, make these appear as a shield icon that if the ship encounters them, provides full sheilding to the ship for 10 seconds.
   - Make the power-ups in the weapons be more impressive.  Every 3 power-ups make the weapon output change appearance/power
   - Clear score display
   - Arrow keys to change direction
   - space bar to shoot
   - Instructions shown on screen

**Save the file:**
   - Use write_file to save as 'rtype_game.html'
   - After saving, respond with confirmation that the game was created
"""

## Run Agent with MCP Tools

In [ ]:
import json
import os
import aisuite as ai

# --- CONFIGURATION ---
# Using your preferred Sonnet 4 model with the required 'us.' prefix
#model = "aws:us.anthropic.claude-sonnet-4-20250514-v1:0"
model = "aws:us.anthropic.claude-sonnet-4-5-20250929-v1:0"
target_filename = "rtype_game.html"
absolute_file_path = os.path.join(os.getcwd(), target_filename)

client = ai.Client()

# KEEPING THE FETCH TOOL (but excluding filesystem to avoid errors)
# tools = fetch_mcp.get_callable_tools() 
# (Note: Use the variable name you initialized in your Setup cell)
tools = fetch_mcp.get_callable_tools() if 'fetch_mcp' in globals() else []

print(f"Starting R-Type Game Agent with {model}...")
print(f"Target Save Location: {absolute_file_path}")

# Prompt forces the model to output the HTML directly instead of trying to save it
final_prompt = f"""{prompt}

IMPORTANT: Do NOT use any tools to save files. 
If you need to fetch external data, use your tools. 
Otherwise, provide the FULL, complete code for '{target_filename}' inside a markdown code block (```html ... ```) as part of your final response.
"""

messages = [{"role": "user", "content": final_prompt}]

# --- EXECUTION LOOP ---
for i in range(10):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools
    )
    
    assistant_msg = response.choices[0].message
    messages.append(assistant_msg)
    
    # Check if the model is done or wants to call a tool (like fetch)
    if not hasattr(assistant_msg, 'tool_calls') or not assistant_msg.tool_calls:
        break
        
    for tool_call in assistant_msg.tool_calls:
        tool_func = next(t for t in tools if t.__name__ == tool_call.function.name)
        args = json.loads(tool_call.function.arguments)
        raw_result = tool_func(**args)
        
        # Bedrock Fix: Ensure tool results are JSON objects, not bare strings
        if isinstance(raw_result, str):
            try:
                final_result = json.loads(raw_result)
            except:
                final_result = {"output": raw_result}
        else:
            final_result = raw_result

        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "name": tool_call.function.name,
            "content": json.dumps(final_result) 
        })

# --- POST-PROCESSING: MANUAL FILE WRITE ---
final_text = getattr(assistant_msg, 'content', "")

if "```html" in final_text:
    html_code = final_text.split("```html")[1].split("```")[0].strip()
    
    # Native Python write always works in WSL
    with open(absolute_file_path, 'w') as f:
        f.write(html_code)
    
    print(f"✅ SUCCESS: Python manually wrote the file to {absolute_file_path}")
else:
    print("❌ ERROR: The model didn't provide an HTML code block.")

print(f"\nModel Summary:\n{final_text[:500]}...")

## Play the Game

In [ ]:
if os.path.exists('rtype_game.html'):
    display(IFrame(src='rtype_game.html', width=600, height=800))
    print("\n💡 Open 'rtype_game.html' in your browser for full view")
else:
    print("⚠️ Game not created. Printing response from the model:")
    print(f"\n{response.choices[0].message.content}")

In [ ]:
print(f"\n{response.choices[0].message.content}")

## Cleanup

In [ ]:
filesystem_mcp.close()
print("✓ Done!")

---

## That's It!

In just a few lines of code, you had an AI:
- ✅ Generate a complete R-Type game from scratch
- ✅ Save it to disk using MCP filesystem tools
- ✅ Display it playable right in the notebook

**Try it yourself:**
- Ask for a different game (Pong, Tetris, etc.)
- Add difficulty levels or speed settings
- Request different color themes
- Try with different models (swap the commented line)